# Programación Evolutiva

In [16]:
import numpy as np 
import math
import copy

Definimos la función a optimizar. En este caso es la función de Ackley

In [17]:
def f(x):
    sumatoria1 = 0
    sumatoria2 = 0
    for x_ in x:
        sumatoria1 += x_**2
        sumatoria2 += math.cos(x_ * math.pi * 2)
    p1 = -20*(math.e**(-0.2*(math.sqrt(((1/len(x)*sumatoria1))))))
    p2 = -math.e**(((1/len(x))*sumatoria2))
    
    return p1 + p2 + 20 + math.e

Ahora definimos la adecuación que es lo que nos va decir qué tan buena es la solución. En este caso entre más pequeña sea f(x), será mejor pues queremos minimizar. Por lo tanto la adecuación se puede definir como menos f(x) y así la mejor solución va a ser la que se encuentre al principio al momento de ordenarlas de mayor a menor.

In [18]:
def adecuacion(x):
    return -f(x)

Para la función de la población inicial tomamos como parámetros la cantidad de padres que queremos inicialmente (mu) y la cantidad de variables que va a tener cada padre o solución (n)

In [19]:
def poblacion_inicial(mu, n):
    '''n es la cantidad de valores de una solución 
        ej [x1,x2,x3,s1,s2,s3] tiene 3 soluciones'''
    padres = []
    for _ in range(mu):
        x = list(np.random.uniform(-30, 30, n))
        sigmas = list(np.random.uniform(0, 1, n))
        # p = list(zip(np.random.uniform(-30, 30, n), np.random.uniform(0, 1, n)))
        p = [x, sigmas]
        p.append(adecuacion(p[0]))
        padres.append(p)
    return padres

Para la función de mutación queremos modificar un poco las soluciones padres y los factores de mutación (sigma) y regresamos el padre modificado

In [20]:
def mutacion(padre, n, alpha, epsilon):
    hijo = copy.deepcopy(padre)
    # hijo[0][0], hijo[0][1] = list(hijo[0][0]), list(hijo[0][1])
    # print(hijo)
    for i in range(len(hijo[1])):
        hijo[1][i] = alpha*np.random.normal(0,1,1)[0]
        if hijo[1][i] < epsilon:
            hijo[1][i] = epsilon
    for i in range(len(hijo[0])):
        hijo[0][i] = hijo[0][i] + hijo[1][i]*np.random.normal(0,1,1)[0]
        if hijo[0][i] <-30:
            hijo[0][i] = -30
        elif hijo[0][i] > 30:
            hijo[0][i] = 30
    # print(hijo[0])
    hijo[2] = adecuacion(hijo[0])
    return hijo

Para la función evolución tomamos la cantidad de generaciones en las cuales vamos a querer estar mutando a la población inicial y eligiendo a los más adecuados. Al final nos quedamos con la solución que tenga mejor adecuación después de todas las generaciones.

In [21]:
def evolucion(n, mu, generaciones, alpha, epsilon = 0.01):
    padres = poblacion_inicial(mu, n)

    for _ in range(generaciones):
        generacion = copy.deepcopy(padres)
        for p in padres:
            hijo = mutacion(p, n, alpha, epsilon)
            generacion.append(hijo)
        generacion = sorted(generacion, key= lambda x: x[-1])
        seleccion = generacion[mu:]
        padres = copy.deepcopy(seleccion)

    return padres[-1]

 Algoritmo evolutivo con tres variables 

In [22]:
evolucion(3,100,2000,0.2,0.005)

[[1.702191010251911e-05, -0.00023318266459686687, 0.00011928504091237896],
 [0.005, 0.005, 0.005],
 -0.0006073817114082836]

En este caso se obtuvo una adecuación de -0.0006, la cual es muy buena pues representa que f(x) = 0.0005 y sabemos que el mínimo global se encuentra en (0,0), por lo cual fue una muy buena aproximación

## Parte 2

In [23]:
import statistics

Para determinar la mejor solución, la peor y la mediana lo que hacemos es ordenar la lista de soluciones obtenidas de acuerdo a su adecuación. En el caso de la mejor solución lo que hacemos es regresar el último elemento, para la peor regresamos el primer elemento y para la mediana regresamos el elemento de en medio

In [24]:
def get_best_solution(solutions):
    best = sorted(solutions, key= lambda x: x[-1])
    return best[-1]

def get_worst_solution(solutions):
    best = sorted(solutions, key= lambda x: x[-1])
    return best[0]

def get_median(solutions):
    best = sorted(solutions, key= lambda x: x[-1])
    return best[len(solutions)//2]

Para el promedio se suman todas la adecuaciones y se dividen entre la cantidad de soluciones que se generaron

In [25]:
def get_mean_cost(solutions):
    cont = 0
    for s in solutions:
        cont += -s[-1]
    return cont/len(solutions)

Para la desviación estándar se separan las adecuaciones y se utiliza la función de statistics.stdev para calcular la desviación estándar de estas.

In [26]:
def get_sd_cost(solutions):
    valores = []
    for s in solutions:
        valores.append(s[-1])
    return statistics.stdev(valores)

Reporte de soluciones para 50 ejecuciones con 3 variables

In [27]:
m = 50
sols = []
for i in range(m):
    solucion = evolucion(3,100,500,0.2,0.005)
    sols.append(solucion)

print('*************************************** Mejor Solución ***************************************')
mejor_sol = get_best_solution(sols)
print(f'Xs = {mejor_sol[0]}')
print(f'Sigmas = {mejor_sol[1]}')
print(f'f(x) = {-mejor_sol[2]}')
print('\n')

print('*************************************** Peor Solución ***************************************')
peor_sol = get_worst_solution(sols)
print(f'Xs = {peor_sol[0]}')
print(f'Sigmas = {peor_sol[1]}')
print(f'f(x) = {-peor_sol[2]}')
print('\n')

print('******************************************* Mediana *******************************************')
med_sol = get_median(sols)
print(f'Xs = {med_sol[0]}')
print(f'Sigmas = {med_sol[1]}')
print(f'f(x) = {-med_sol[2]}')
print('\n')

print('Promedio:\t',get_mean_cost(sols))
print('Stadard Dev:\t',get_sd_cost(sols))

*************************************** Mejor Solución ***************************************
Xs = [-4.927451080863067e-06, -2.7862473673831846e-05, -0.00010752772624494011]
Sigmas = [0.005, 0.005, 0.005]
f(x) = 0.00025699754687780185


*************************************** Peor Solución ***************************************
Xs = [0.0005492624654099324, -0.0004007344186573653, 0.0001925155525000411]
Sigmas = [0.005, 0.005, 0.005]
f(x) = 0.0016407787029462106


******************************************* Mediana *******************************************
Xs = [-0.00035532122022141117, -0.00018800042915853078, 0.00010866182354495545]
Sigmas = [0.005, 0.005, 0.005]
f(x) = 0.0009647566390644791


Promedio:	 0.0009698356586852875
Stadard Dev:	 0.00031159834712914624


### Reportar resultados para 5, 10 y 20 variables

Primero para 5 variables y 200 ejecuciones con: mu = 100, 500 generaciones, alpha = 0.2 y epsilon = 0.005

In [28]:
m = 200
sols = []
for i in range(m):
    solucion = evolucion(5,100,500,0.2,0.005)
    sols.append(solucion)

print('*************************************** Mejor Solución ***************************************')
mejor_sol = get_best_solution(sols)
print(f'Xs = {mejor_sol[0]}')
print(f'Sigmas = {mejor_sol[1]}')
print(f'f(x) = {-mejor_sol[2]}')
print('\n')

print('*************************************** Peor Solución ***************************************')
peor_sol = get_worst_solution(sols)
print(f'Xs = {peor_sol[0]}')
print(f'Sigmas = {peor_sol[1]}')
print(f'f(x) = {-peor_sol[2]}')
print('\n')

print('******************************************* Mediana *******************************************')
med_sol = get_median(sols)
print(f'Xs = {med_sol[0]}')
print(f'Sigmas = {med_sol[1]}')
print(f'f(x) = {-med_sol[2]}')
print('\n')

print('Promedio:\t',get_mean_cost(sols))
print('Stadard Dev:\t',get_sd_cost(sols))

*************************************** Mejor Solución ***************************************
Xs = [-0.0004264127154008789, 7.934136018297299e-05, 0.00022985196998591634, 0.00044495752571934236, -0.0006481202840276234]
Sigmas = [0.005, 0.005, 0.005, 0.005, 0.0056135841093209946]
f(x) = 0.0016671018659661918


*************************************** Peor Solución ***************************************
Xs = [-3.950253574110077, -5.978580103343751, 1.0041343461516834, -6.936038825150418, -4.954966728888865]
Sigmas = [0.005, 0.22940202883673277, 0.3723953683112738, 0.005, 0.18343952320548304]
f(x) = 12.737583456622763


******************************************* Mediana *******************************************
Xs = [-0.0022355513104083727, 0.0011415430464496177, 0.0015310090273575883, 0.0007423277611596588, 0.0012729127479682596]
Sigmas = [0.005, 0.005, 0.005, 0.005, 0.005]
f(x) = 0.005998373895199638


Promedio:	 0.3213612849784269
Stadard Dev:	 1.6705049725513619


Ahora para 10 variables y 200 ejecuciones con: mu = 100, 500 generaciones, alpha = 0.2 y epsilon = 0.005

In [29]:
m = 200
sols = []
for i in range(m):
    solucion = evolucion(10,100,500,0.2,0.005)
    sols.append(solucion)

print('*************************************** Mejor Solución ***************************************')
mejor_sol = get_best_solution(sols)
print(f'Xs = {mejor_sol[0]}')
print(f'Sigmas = {mejor_sol[1]}')
print(f'f(x) = {-mejor_sol[2]}')
print('\n')

print('*************************************** Peor Solución ***************************************')
peor_sol = get_worst_solution(sols)
print(f'Xs = {peor_sol[0]}')
print(f'Sigmas = {peor_sol[1]}')
print(f'f(x) = {-peor_sol[2]}')
print('\n')

print('******************************************* Mediana *******************************************')
med_sol = get_median(sols)
print(f'Xs = {med_sol[0]}')
print(f'Sigmas = {med_sol[1]}')
print(f'f(x) = {-med_sol[2]}')
print('\n')

print('Promedio:\t',get_mean_cost(sols))
print('Stadard Dev:\t',get_sd_cost(sols))

*************************************** Mejor Solución ***************************************
Xs = [-0.011041913023473713, 0.002126623690886195, 0.0026082584753518833, -0.013130161262573298, -0.017866634903470843, -0.011824772899578527, 0.03594501667291706, 0.0024443652897363647, -0.0075033329834319085, 0.006851644355829427]
Sigmas = [0.005, 0.0942606652108702, 0.005, 0.12533660798420518, 0.005, 0.005, 0.005, 0.005, 0.005, 0.009740813430526421]
f(x) = 0.07034690355359574


*************************************** Peor Solución ***************************************
Xs = [1.002346784874123, -10.002863295476965, 2.9844702898195217, 16.998753680725176, -19.994057826809534, 9.01328499730173, 5.0178728625848645, 26.0203638175763, 6.003045735911485, 4.009626392965903]
Sigmas = [0.005, 0.005, 0.005, 0.024342652455460148, 0.005, 0.005, 0.005, 0.005, 0.04242440387886649, 0.0229635113388476]
f(x) = 18.45590085118775


******************************************* Mediana *************************

Ahora para 20 variables y 150 ejecuciones con: mu = 100, 500 generaciones, alpha = 0.2 y epsilon = 0.005

In [31]:
m = 150
sols = []
for i in range(m):
    solucion = evolucion(20,100,500,0.2,0.005)
    sols.append(solucion)

print('*************************************** Mejor Solución ***************************************')
mejor_sol = get_best_solution(sols)
print(f'Xs = {mejor_sol[0]}')
print(f'Sigmas = {mejor_sol[1]}')
print(f'f(x) = {-mejor_sol[2]}')
print('\n')

print('*************************************** Peor Solución ***************************************')
peor_sol = get_worst_solution(sols)
print(f'Xs = {peor_sol[0]}')
print(f'Sigmas = {peor_sol[1]}')
print(f'f(x) = {-peor_sol[2]}')
print('\n')

print('******************************************* Mediana *******************************************')
med_sol = get_median(sols)
print(f'Xs = {med_sol[0]}')
print(f'Sigmas = {med_sol[1]}')
print(f'f(x) = {-med_sol[2]}')
print('\n')

print('Promedio:\t',get_mean_cost(sols))
print('Stadard Dev:\t',get_sd_cost(sols))

*************************************** Mejor Solución ***************************************
Xs = [-2.0174424968609657, 0.011191668212970188, 6.965117817867044, 0.9780887963682791, 16.021539381856314, -14.998909340007213, -5.975650052962269, 12.944131640757922, -10.927060457558696, -0.9136772203992282, 15.95172761037061, -11.99887959419019, 6.0168367926999355, -10.993344473329765, 3.025672157248305, 20.00808015986183, -7.033795375178076, 7.977770895051629, 2.879381055552799, 7.017830428902072]
Sigmas = [0.005, 0.07206731370955509, 0.16265612558406162, 0.005, 0.04490563412350569, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.18040022782257809, 0.005, 0.005, 0.005, 0.005, 0.006185173199598133, 0.11471462830435662, 0.005]
f(x) = 17.38549730142167


*************************************** Peor Solución ***************************************
Xs = [0.9940961393570537, 24.052711314283467, 14.987163162456996, -25.062115640045352, 13.016904132671806, -0.020772330640955697, 8.96817629266